In [1]:
import numpy as np
import laspy
from TronGisPy import GisIO, CRS, Normalizer, Interpolation
import CSF 
import os, sys, gc, subprocess # file system package
import pickle, json, shutil, itertools # file processing basic package
import laspy, PySaga, gdal # geo package
#import cv2 # open computer vision package
import pandas as pd
import geopandas as gpd
import numba
# csf = CSF.CSF()
from scipy.stats import entropy
from matplotlib import pyplot as plt
from TronGisPy import GisIO, CRS, Normalizer, Interpolation
from shapely.geometry import Point, Polygon, MultiPolygon
from sklearn.metrics.pairwise import euclidean_distances, pairwise_distances, pairwise_distances_argmin
from util.las import *

In [2]:
%%time

las_fp = os.path.join("data", "123.las")
X_ref_fp = "./tmp/X_ref.tif"
las_classified_path = "./tmp/classified_ground.las"
street_las = CloudPoint(las_fp)

Wall time: 4.17 s


In [3]:
resolution = 0.05
pad = resolution / 2
x_min, x_max, y_min, y_max = street_las.get_las_boundary(return_poly=False)
x_min, x_max = x_min - pad, x_max + pad
y_min, y_max = y_min - pad, y_max + pad
geo_transform = (x_min, resolution, 0, y_max, 0, -resolution)
wkt_3826 = CRS.get_wkt_from_epsg(3826)
cols = int((x_max - x_min) // resolution) + 1
rows = int((y_max - y_min) // resolution) + 1
X = np.random.rand(rows, cols)
GisIO.write_output_tif(X, X_ref_fp, geo_transform=geo_transform, projection=wkt_3826, gdaldtype=gdal.GDT_Float32)
X_ref = GisIO.get_nparray(X_ref_fp)
print(X_ref.shape)

(3053, 3256, 1)


In [4]:
%%time


las_data = np.array([street_las.data[i] for i in ["x", "y"]]).T
X_ref = GisIO.get_nparray(X_ref_fp)
cols, rows, bands, _, _, _, _ = GisIO.get_geo_info(X_ref_fp)
X_ref_extent = GisIO.get_extent(X_ref_fp, False)

las_npidxs = CRS.numba_transfer_group_coord_to_npidx(las_data, geo_transform) # Wall time: 35 s
las_npidxs

Wall time: 30.1 s


array([[2409,  769],
       [2480,  771],
       [2490,  785],
       ...,
       [1408, 2321],
       [1487, 2468],
       [1383, 2480]], dtype=int64)

In [5]:
%%time
csf = CSF.CSF()
csf.params.bSloopSmooth = False
csf.params.cloth_resolution = 0.5
csf.setPointCloud(np.array([street_las.data[i] for i in ["x", "y", "z"]]).T)
ground = CSF.VecInt()  # a list to indicate the index of ground points after calculation
non_ground = CSF.VecInt() # a list to indicate the index of non-ground points after calculation
csf.do_filtering(ground, non_ground) # do actual filtering.
scale = street_las.header[1]



Wall time: 45.6 s


In [6]:
%%time
def write_las_data(las_data, dst_las_fp, scale=[1.0,1.0,1.0]):
    x, y, z, num_returns, return_num, intensity, classification, red, green, blue = las_data.T
    offset = [np.floor(np.min(x)), np.floor(np.min(y)), np.floor(np.min(z))]

    outFile = laspy.file.File(dst_las_fp, mode="w", header=laspy.header.Header(point_format=2)) # LAS point format 2 supports color
    outFile.header.offset, outFile.header.scale = offset, scale
    
    outFile.x, outFile.y, outFile.z =x, y, z
    outFile.num_returns, outFile.return_num = num_returns.astype(np.int), return_num.astype(np.int)
    outFile.intensity = intensity
    outFile.classification = classification.astype(np.int)
    outFile.red, outFile.green, outFile.blue = red, green, blue
    outFile.close()

ground_las_data = np.array([street_las.data[i] for i in street_las.using_attr]).T
ground_las_data = ground_las_data[ground]
# las_data[ground] = 2 # groud point
# las_data[non_ground] = 1 # unassigned
write_las_data(ground_las_data, las_classified_path, scale)

Wall time: 26.1 s


In [7]:
%%time


las_data = np.array([street_las.data[i] for i in street_las.using_attr]).T
X_ref = GisIO.get_nparray(X_ref_fp)
cols, rows, bands, _, _, _, _ = GisIO.get_geo_info(X_ref_fp)
X_ref_extent = GisIO.get_extent(X_ref_fp, False)

las_npidxs = CRS.numba_transfer_group_coord_to_npidx(las_data, geo_transform) # Wall time: 35 s
las_npidxs, las_npidxs.shape

Wall time: 35.1 s


(array([[2409,  769],
        [2480,  771],
        [2490,  785],
        ...,
        [1408, 2321],
        [1487, 2468],
        [1383, 2480]], dtype=int64),
 (23628925, 2))

In [ ]:
%%time


las_data = np.array([cli.data[i] for i in street_las.using_attr]).T
X_ref = GisIO.get_nparray(X_ref_fp)
cols, rows, bands, _, _, _, _ = GisIO.get_geo_info(X_ref_fp)
X_ref_extent = GisIO.get_extent(X_ref_fp, False)

las_npidxs = CRS.numba_transfer_group_coord_to_npidx(las_data, geo_transform) # Wall time: 35 s
las_npidxs, las_npidxs.shape